In [ ]:
from myfunctions import get_keywords,get_dico_keywords, get_cos_sim, get_keyword,clean_text
import pandas as pd
import pickle
from tqdm import tqdm
import math

In [ ]:
myyears = [2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022]
mymonths = ['January', 'February', 'March', 'April', 'May', 'June', 'July',\
           'August', 'September', 'October', 'November', 'December']

In [ ]:
# I do a list of paper such that I can count every paper only once

infile_filtered_data = open('data_creation_variables/filtered_data','rb')
filtered_data = pickle.load(infile_filtered_data)
infile_filtered_data.close()

In [ ]:
infile_data_full = open('../exploratory_analysis/data_exploratory_analysis/df_full_cleaned','rb')
df_full = pickle.load(infile_data_full)
infile_data_full.close()

In [ ]:
items = ['paper','keyword','cosine_similarity','publication_date','year','month']

In [ ]:
dicokeywords = {'paper':[],
                'keyword':[],
                'cosine_similarity':[],
                'publication_date':[],
                'year':[],
                'month':[]}

In [ ]:
df_full = df_full[['paper','title','publication_date','year','month','abstract']]
df_full = df_full.drop_duplicates()

In [ ]:
listpaper_first = list(set(df_full.paper.tolist()))

In [ ]:
dicokeywords = {'paper': [],
                    'keyword': [],
                    'cosine_similarity': [],
                    'publication_date': [],
                    'year': [],
                    'month': []}

In [ ]:
fractioning_number = 100

In [ ]:
for j in range(5):
    list_listkeywords = []
    list_listcosim = []
    step = int(math.floor(len(listpaper_first)/ 5))
    start = j * step
    if j == 4:
        end = len(listpaper_first)
    else:
        end = (j + 1) * step
    listpaper = listpaper_first[start:end]
    for i in tqdm(range(fractioning_number)):
        step = int(math.floor(len(listpaper) / fractioning_number))
        start = i*step
        end = (i+1)*step
        if i == fractioning_number-1:
            end = len(listpaper)
        restricted_listpaper=listpaper[start:end]
        mypapers =df_full[df_full['paper'].isin(restricted_listpaper)].copy()
        for paper in restricted_listpaper:
            myinfos = mypapers.loc[mypapers['paper']==paper].copy()
            abstract = list(set(myinfos.abstract.tolist()))
            title = list(set(myinfos.title.tolist()))
            text = str(title[0] + abstract[0])

            text_c = clean_text(text)

            # this functions get_keyword take 99.99% of the running time for each loop
            mykeywords = get_keyword(text_c)

            numberkeywords = len(mykeywords)

            # actually mykeywords is a list of tuples
            # I take every element of the list and then I take the
            # elements of the tuple I am interested in.
            listkeywords = []
            listcosim=[]
            for y in range(numberkeywords):
                infomykey = mykeywords[y]
                listkeywords.append(infomykey[0])
                listcosim.append(infomykey[1])
            list_listkeywords.append(listkeywords)
            list_listcosim.append(listcosim)
            # saving my list of author for further purposes
    with open("data_creation_variables/list_listkeywords"+str(j), "wb") as fp:  # Pickling
        pickle.dump(list_listkeywords, fp)
    with open("data_creation_variables/list_listcosim"+str(j), "wb") as fp:  # Pickling
        pickle.dump(list_listcosim, fp)

In [ ]:
list_keywords = []
list_cosim = []
for j in range(5):
    with open("data_creation_variables/list_listkeywords"+str(j), 'rb') as f:
        list_keywords=list_keywords+pickle.load(f)
    with open("data_creation_variables/list_listcosim"+str(j), 'rb') as f:
        list_cosim = list_cosim + pickle.load(f)

In [ ]:
df_full = df_full[['paper','publication_date','year','month']]

In [ ]:
def get_publication_date(paper,df_full):
    df = df_full.loc[df_full['paper']==paper].copy()
    listinfo = df.publication_date.tolist()
    return listinfo[0]

In [ ]:
def get_year(paper,df_full):
    df = df_full.loc[df_full['paper']==paper].copy()
    listinfo = df.year.tolist()
    return listinfo[0]

In [ ]:
def get_month(paper,df_full):
    df = df_full.loc[df_full['paper']==paper].copy()
    listinfo = df.month.tolist()
    return listinfo[0]

In [ ]:
publication_list = list(map(lambda x: get_publication_date(x, df_full), tqdm(listpaper_first)))
month_list = list(map(lambda x: get_month(x, df_full), tqdm(listpaper_first)))
year_list = list(map(lambda x: get_year(x, df_full), tqdm(listpaper_first)))

In [ ]:
dicokeywords['keyword']=list_keywords
dicokeywords['cosine_similarity']= list_cosim
dicokeywords['paper']=listpaper_first
dicokeywords['publication_date']=publication_list
dicokeywords['year']=year_list
dicokeywords['month']=month_list

In [ ]:
dfkeywords = pd.DataFrame(dicokeywords)

In [ ]:
dfkeywords =dfkeywords.explode(['keyword', 'cosine_similarity'])

In [ ]:
dfkeywords.to_pickle('data_creation_variables/dfkeywords')